In [1]:
#imports
!pip install python-docx
!pip install docx2txt
!pip install dnspython
!pip install pymongo
import docx
import docx2txt
import sqlalchemy
import pandas as pd
import os
from pymongo import MongoClient
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#Connect to remote MongoDB database hosted on Heroku
def get_database():

    # Provide the mongodb atlas url to connect python to mongodb using pymongo
    CONNECTION_STRING = "mongodb+srv://apprenticeship:" + "insertpassword" + "@kmsdataextraction.jodxa.mongodb.net/?retryWrites=true&w=majority"

    # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
    client = MongoClient(CONNECTION_STRING)

    # Create the database for our example (we will use the same database throughout the tutorial
    return client['data-extraction']

extraction_db = get_database()

companies = extraction_db.companies
questions = extraction_db.questions

In [6]:
#Extract raw text from example docx file and remove apostrophes so SQL insertions work
!wget https://github.com/rohanchanani/ApprenticeshipKMS/blob/main/Completed_Anonym_KMS_Assessment_Discussion_Framework_R03.3-.docx?raw=true
!mv /content/Completed_Anonym_KMS_Assessment_Discussion_Framework_R03.3-.docx?raw=true /content/Completed_Anonym_KMS_Assessment_Discussion_Framework_R03.3-.docx
text = docx2txt.process("/content/Completed_Anonym_KMS_Assessment_Discussion_Framework_R03.3-.docx")
all_lines = [line.strip().replace("'", "") for line in text.split("\n") if line != ""]

--2022-06-02 18:03:51--  https://github.com/rohanchanani/ApprenticeshipKMS/blob/main/Completed_Anonym_KMS_Assessment_Discussion_Framework_R03.3-.docx?raw=true
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/rohanchanani/ApprenticeshipKMS/raw/main/Completed_Anonym_KMS_Assessment_Discussion_Framework_R03.3-.docx [following]
--2022-06-02 18:03:51--  https://github.com/rohanchanani/ApprenticeshipKMS/raw/main/Completed_Anonym_KMS_Assessment_Discussion_Framework_R03.3-.docx
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rohanchanani/ApprenticeshipKMS/main/Completed_Anonym_KMS_Assessment_Discussion_Framework_R03.3-.docx [following]
--2022-06-02 18:03:51--  https://raw.githubusercontent.com/rohanchanani/ApprenticeshipKMS/main/Completed_Anonym_KMS_Assess

In [8]:
#Extract question and section lines from docx file based on paragraph formatting. Sections are "Heading 1" and questions are "Heading 2" or "Heading 3"
doc = docx.Document("/content/Completed_Anonym_KMS_Assessment_Discussion_Framework_R03.3-.docx")
found_questions = [paragraph.text.strip().replace("'", "") for paragraph in doc.paragraphs if paragraph.text != "" and paragraph.style.name in ["Heading 2", "Heading 3"]]
sections = [paragraph.text.strip().replace("'", "") for paragraph in doc.paragraphs if paragraph.text != "" and paragraph.style.name == "Heading 1"]

In [ ]:
#Given database and question data, insert a line into the question table for each new question and a line into the company table for each new line
def upload_question_to_database(question, answer, section, company, version, number, question_results, company_results):   
    if not question_results.get(question+answer):
        questions.insert_one({"number":number, "question":question, "answer":answer, "version": version, "company":company, "section":section})

In [ ]:
def read_questions(company, version, current_section, i, all_lines, found_questions, sections):
    #Cached duplicates
    question_results = {test["question"]+test["answer"]:1 for test in list(questions.find({}, {"_id":0}))}
    company_results = {test["company_name"]:1 for test in list(companies.find({}, {"_id":0}))}
    
    #Match questions with answers
    question_number = 1
    while i < len(all_lines):
        current_question = all_lines[i]
        try:
            next_ten_questions = found_questions[found_questions.index(current_question) + 1: found_questions.index(current_question) + 11]
        except:
            i += 1
            continue
        current_answer = all_lines[i+1]
        i += 2
        while i < len(all_lines):
            if all_lines[i] in next_ten_questions:
                break
            if all_lines[i] == sections[min(sections.index(current_section) + 1, len(sections) - 1)]:
                current_section = all_lines[i]
                i += 1
                break
            current_answer += "\n" + all_lines[i]
            i += 1
        upload_question_to_database(current_question, current_answer, current_section, company, version, question_number, question_results, company_results)
        question_number += 1

In [ ]:
#Read the full document

#Counter to track current line of document
i = 0
version_not_found = True
current_section = ""
questions_started = False
while not questions_started:
    
    #Find the company name and insert if not duplicate
    if all_lines[i] == "Technology Assessment of":
        company_name = all_lines[i+1]
        if not len(list(companies.find({"company_name":company_name}))):
            companies.insert_one({"company_name":company_name})
        i += 2
    #Find the company ADF version
    elif "Document Version" in all_lines[i] and version_not_found:
        document_version = all_lines[i].split(" ")[-1]
        version_not_found = False
        i += 1
    
    #Find the first section and start the questions
    elif all_lines[i] in sections:
        current_section = all_lines[i]
        i += 1
        questions_started = True
    else:
        i += 1
read_questions(company_name, document_version, current_section, i, all_lines, found_questions, sections)